<a href="https://colab.research.google.com/github/lustea0201/Interpretability/blob/master/Interpretation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting the outputs of the model on the second dataset

In [1]:
#@title
import numpy as np
import torch 
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
from torch.utils.data import DataLoader
from google.colab import drive
drive.mount('/content/drive/')

!pip install flashtorch
from flashtorch.saliency import Backprop

!pip install shap 
import shap

!pip install pytorch-gradcam
from torchvision import transforms
from torchvision.utils import make_grid, save_image

from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp

!wget https://raw.githubusercontent.com/yiskw713/SmoothGradCAMplusplus/master/cam.py -P local_modules -nc
!wget https://raw.githubusercontent.com/yiskw713/SmoothGradCAMplusplus/master/utils/visualize.py -P local_modules -nc
import sys
sys.path.append('local_modules')
from PIL import Image
from local_modules.visualize import visualize
import local_modules.cam as smooth

!wget https://raw.githubusercontent.com/lustea0201/Interpretability/master/HierarchicalShapley.py -P local_modules -nc
import local_modules.HierarchicalShapley as HS

!wget https://raw.githubusercontent.com/lustea0201/Interpretability/master/utils.py -P local_modules -nc
import local_modules.utils as utils

!wget https://raw.githubusercontent.com/lustea0201/Interpretability/master/interpret11.py -P local_modules -nc
import local_modules.interpret11 as intp

torch.manual_seed(0)
dtype = torch.float

FIGSIZE = (5,4)

net = utils.Net() # Model instantiation
# Loading the trained dictionnary state
net.load_state_dict(torch.load('drive/My Drive/Interpretability/model2.pth')) 

data = zipfile.ZipFile("/content/drive/My Drive/Interpretability/5000/data2/data.zip", 'r')

root_dir = "main_dir"
data.extractall(root_dir)
data.close()

Batch_Size = 64
#transf_temp =  transforms.Compose( [ToTensor()])
#train_data_temp = ImageFolder(root = os.path.join(root_dir, 'train'), transform = transf_temp)
#dataloader_temp = DataLoader(train_data_temp, batch_size = Batch_Size, shuffle = True, num_workers = 0)
#MEAN, STD = utils.datasetMeanStd(dataloader_temp)
MEAN, STD = np.array([0.5, 0.5, 0.5]), np.array([0.5, 0.5, 0.5])

transf = transforms.Compose( [ToTensor(), Normalize(mean=MEAN, std=STD)])

train_data = ImageFolder(root = os.path.join(root_dir, 'train'), transform = transf)
dataloader = DataLoader(train_data, batch_size = Batch_Size, shuffle = True, num_workers = 0)
train_loader = iter(dataloader)


ImF = ImageFolder(root = "/content/drive/My Drive/Interpretability/img4saliencymap/data2", transform = transf)
batch_Size = 3
exloader = DataLoader(ImF, batch_size = batch_Size, shuffle = False, num_workers = 0)

exIter = iter(exloader)
images, labels = next(exIter)

# For guided backprop
backprop = Backprop(net)

# For SHAP
X,Y = next(train_loader)
bg_choice = "average" # Default: entire training set, "average", "white", "black" 
# Median would result in white 
background = X
if (bg_choice == "black"): 
  background = -torch.ones(X.shape)
elif (bg_choice == "white"): 
  background = torch.ones(X.shape)
elif (bg_choice == "average"): 
  avg = torch.mean(X, dim = 0)
e = shap.GradientExplainer(net, background)

# For GradCAM
gradcam_conv = GradCAM(net, net.conv2)
gradcam_pp_conv = GradCAMpp(net, net.conv2)
gradcam_pool = GradCAM(net, net.pool2)
gradcam_pp_pool = GradCAMpp(net, net.pool2)

# For SmoothCAM
target_layer = net.conv2
wrapped_G = smooth.GradCAM(net, target_layer)
wrapped_P = smooth.GradCAMpp(net, target_layer)
wrapped_S = smooth.SmoothGradCAMpp(net, target_layer, n_samples=25, stdev_spread=0.15)
wrapped = [wrapped_G, wrapped_P, wrapped_S]

# For Hierarchical 
average = torch.mean(X, dim = 0)
h = HS.HierarchicalShap(net, background = average)

ModuleNotFoundError: No module named 'seaborn'

In [2]:
h.saliency_map(images[1], labels[1], tolerance = [2,3,4,5,6], only_one_run = False, max_depth = 30, debug = False)

NameError: name 'h' is not defined


# EXAMPLE 0

In [3]:
image0 = images[0]
label0 = labels[0]

input0 = image0.view(-1, 3, 100, 120) # This shape is necessary for the network 
output0 = net(input0)
_, predicted0 = torch.max(output0.data, 1)

img0 = utils.input2image(image0, MEAN, STD)
utils.display_image(img0, label0.numpy(), int(predicted0))

NameError: name 'images' is not defined

### Guided backpropagation explanation

In [4]:
input0.requires_grad = True # Necessary to compute the gradient
gradients0 = backprop.calculate_gradients(input0, label0, guided = True).detach().numpy()
intp.display_gradients(gradients0, FIGSIZE).suptitle("Guided backpropagation across the different channels for an image with label 0", size="xx-large");
backprop.visualize(input0, label0, guided=True, figsize = (3*FIGSIZE[0], FIGSIZE[1]), cmap="bwr")

NameError: name 'input0' is not defined

### GradCAM explanation

In [5]:
intp.gradcam_exp(gradcam_conv, gradcam_pp_conv, input0, img0,  "Second convolutional layer", FIGSIZE)
intp.gradcam_exp(gradcam_pool, gradcam_pp_pool, input0, img0,  "Second pooling layer", FIGSIZE)

NameError: name 'intp' is not defined

### SmoothCAM explanation

In [6]:
intp.smooth_exp(input0, image0, wrapped)

NameError: name 'intp' is not defined

### Shapley explanation

In [7]:
intp.shap_exp(e, input0.detach(), img0)

NameError: name 'intp' is not defined

In [8]:
e2 = shap.DeepExplainer(net, background)


shap_values = e2.shap_values(input0.detach())
shapley_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]

image00 = img0[np.newaxis, :]

shap.image_plot(shapley_values, image00,  show=False)
plt.suptitle("All channels together");


NameError: name 'shap' is not defined

### Hierarchical Shapley explanation

In [9]:
h.saliency_map(image0, label0, tolerance  = [5,5.5,6,6.5,7])
h.saliency_map(image0, label0, tolerance = [6], only_one_run = True)


NameError: name 'h' is not defined

# EXAMPLE 1

In [10]:
image1 = images[1]
label1 = labels[1]

input1 = image1.view(-1, 3, 100, 120)
output1 = net(input1)
_, predicted1 = torch.max(output1.data, 1)

img1 = utils.input2image(image1, MEAN, STD)
utils.display_image(img1, label1.numpy(), int(predicted1))

NameError: name 'images' is not defined

### Backpropagation

In [11]:
input1.requires_grad = True # Necessary to compute the gradient
gradients1 = backprop.calculate_gradients(input1, label1, guided = True).detach().numpy()
intp.display_gradients(gradients1, FIGSIZE).suptitle("Guided backpropagation across the different channels for an image with label 0", size="xx-large");
backprop.visualize(input1, label1, guided=True, figsize = (3*FIGSIZE[0], FIGSIZE[1]), cmap="bwr")

NameError: name 'input1' is not defined

### GradCAM

In [12]:
intp.gradcam_exp(gradcam_conv, gradcam_pp_conv, input1, img1,  "Second convolutional layer", FIGSIZE)
intp.gradcam_exp(gradcam_pool, gradcam_pp_pool, input1, img1,  "Second pooling layer", FIGSIZE)

NameError: name 'intp' is not defined

### SmoothCAM

In [13]:
intp.smooth_exp(input1, image1, wrapped)

NameError: name 'intp' is not defined

### Shapley explanation

In [14]:
intp.shap_exp(e, input1.detach(), img1)

NameError: name 'intp' is not defined

In [15]:
shap_values = e2.shap_values(input1.detach())
shapley_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]

image11 = img1[np.newaxis, :]

shap.image_plot(shapley_values, image11,  show=False)
plt.suptitle("All channels together");

NameError: name 'e2' is not defined

In [16]:
type(image11)

NameError: name 'image11' is not defined

### Hierarchical Shapley

In [17]:

a = 6.0
f = np.linspace(0.7, 1.3, 30)
l = (f*a).tolist()
h.saliency_map(image1, label1, tolerance = l)
h.saliency_map(image1, label1, tolerance = [6], only_one_run = True)

NameError: name 'h' is not defined

In [18]:
# For Hierarchical 
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



class HierarchicalShap2:
    """
    Explains the salient regions of images according a given network.
    """

    def __init__(self, model, background, mean=np.array([0.5, 0.5, 0.5]), sd=np.array([0.5, 0.5, 0.5])):
        """
        Parameters
        ----------
        model : the model from which you wish to study the decision
        background : used to remove the contribution of non-considered regions when constructing subsets
        mean : the mean used for image normalization (useful for plotting from input)
        sd : the standard deviation used for normalization (useful for plotting from input)
        """
        self.model = model
        self.background = background
        self.mean = mean
        self.sd = sd

    def display_cropped_images(self, images, scores):
        """
        Draw the subsets.
        Parameters
        ----------
        images : all the subsets to draw
        scores : the output score for a class 1
        """
        fig, axs = plt.subplots(4, 4, figsize=(15, 15))
        for i in range(4):
            for j in range(4):
                im = images[4 * i + j].numpy().transpose(1, 2, 0)
                im = im * self.sd + self.mean
                axs[i, j].imshow(im)
                axs[i, j].set_title("#%d score:%f " % (4 * i + j, scores[4 * i + j]))

    def construct_subsets(self, im, s=(0, 0), region_size=(None, None)):
        """
        Construct the subsets of im: all possible image resulting from removing from im the content of 0, 1, 2, 3
        or all 4 quadrants of the region defined by start and region_size .
        Parameters
        ----------
        im : the image from which to extract subsets
        s : the top left pixel coordinates of the region analyzed, a tuple of
        region_size : the size of the region analyzed
        Returns
        --------
        subsets : the list of 16 images
        r_coord : a 2x2 array where each entry is a tuple of tuples; the first indicating the start
                  of the region and the second its size
        """

        m = (s[0] + region_size[0] // 2, s[1] + region_size[1] // 2)
        e = (s[0] + region_size[0], s[1] + region_size[1])

        top_left = (s, (m[0] - s[0], m[1] - s[1]))
        top_right = ((s[0], m[1]), (m[0] - s[0], e[1] - m[1]))
        bottom_left = ((m[0], s[1]), (e[0] - m[0], m[1] - s[1]))
        bottom_right = (m, (e[0] - m[0], e[1] - m[1]))
        r_coord = np.array([[top_left, top_right], [bottom_left, bottom_right]])

        subsets_size = [16, im.shape[0], im.shape[1], im.shape[2]]

        bg = self.background
        # removing 0 features
        im1234 = bg.clone()
        im1234[:, s[0]:e[0], s[1]:e[1]] = im[:, s[0]:e[0], s[1]:e[1]]
        # removing 1 feature
        im234 = im1234.clone()
        im234[:, s[0]:m[0], s[1]:m[1]] = bg[:, s[0]:m[0], s[1]:m[1]]
        im134 = im1234.clone()
        im134[:, s[0]:m[0], m[1]:e[1]] = bg[:, s[0]:m[0], m[1]:e[1]]
        im124 = im1234.clone()
        im124[:, m[0]:e[0], s[1]:m[1]] = bg[:, m[0]:e[0], s[1]:m[1]]
        im123 = im1234.clone()
        im123[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        # removing 2 features
        im34 = im234.clone()
        im34[:, s[0]:m[0], m[1]:e[1]] = bg[:, s[0]:m[0], m[1]:e[1]]
        im24 = im234.clone()
        im24[:, m[0]:e[0], s[1]:m[1]] = bg[:, m[0]:e[0], s[1]:m[1]]
        im23 = im234.clone()
        im23[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        im14 = im134.clone()
        im14[:, m[0]:e[0], s[1]:m[1]] = bg[:, m[0]:e[0], s[1]:m[1]]
        im13 = im134.clone()
        im13[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        im12 = im123.clone()
        im12[:, m[0]:e[0], s[1]:m[1]] = bg[:, m[0]:e[0], s[1]:m[1]]
        # removing 3 features
        im4 = im34.clone()
        im4[:, m[0]:e[0], s[1]:m[1]] = bg[:, m[0]:e[0], s[1]:m[1]]
        im3 = im34.clone()
        im3[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        im2 = im24.clone()
        im2[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        im1 = im14.clone()
        im1[:, m[0]:e[0], m[1]:e[1]] = bg[:, m[0]:e[0], m[1]:e[1]]
        # removing 4
        im_ = bg.clone()

        subsets = torch.zeros(size=subsets_size)
        subsets[0] = im1234
        subsets[1] = im234
        subsets[2] = im134
        subsets[3] = im124
        subsets[4] = im123
        subsets[5] = im34
        subsets[6] = im24
        subsets[7] = im23
        subsets[8] = im14
        subsets[9] = im13
        subsets[10] = im12
        subsets[11] = im4
        subsets[12] = im3
        subsets[13] = im2
        subsets[14] = im1
        subsets[15] = im_

        return subsets, r_coord

    def subset_scores(self, sub, label):
        """
        Compute the scores of each subset input.
        Parameters
        ----------
        sub : the subsets of inputs
        label : the class label - typically 1 -  in which we're interested.
        Returns
        --------
        score : an array of the 16 scores for each input
        """
        outputs = self.model(sub)
        score = outputs[:, label].detach().numpy()
 
        return score

    def shapley_of_quadrants(self, score):
        """
        Return a 2x2 array which contains the Shapley values associated with each quadrant
        Parameters
        ----------
        score : the network evaluation for each subset
        Returns
        --------
        shapley_coefficients : an array of the 16 scores for each input
        """

        phi1 = (score[14] - score[15] + score[0] - score[1]) / 4 \
               + (score[8] - score[11] + score[9] - score[12] + score[10] - score[13]
                  + score[2] - score[5] + score[3] - score[6] + score[4] - score[7]) / 12

        phi2 = (score[13] - score[15] + score[0] - score[2]) / 4 \
               + (score[6] - score[11] + score[7] - score[12] + score[10] - score[14]
                  + score[1] - score[5] + score[3] - score[8] + score[4] - score[9]) / 12

        phi3 = (score[12] - score[15] + score[0] - score[3]) / 4 \
               + (score[5] - score[11] + score[9] - score[14] + score[7] - score[13]
                  + score[2] - score[8] + score[1] - score[6] + score[4] - score[10]) / 12

        phi4 = (score[11] - score[15] + score[0] - score[4]) / 4 \
               + (score[8] - score[14] + score[5] - score[12] + score[6] - score[13]
                  + score[1] - score[7] + score[3] - score[10] + score[2] - score[9]) / 12

        shapley_coefficients = np.array([[phi1, phi2], [phi3, phi4]])
        return shapley_coefficients

    def get_salient_regions(self, shapley_values, tol, regions):
        """
        Determine which of the 4 quadrants are salient, i.e. have Shapley value larger than tol
        Parameters
        ----------
        shapley_values : the Shapley coefficients associated with each quadrant
        tol : the specified tolerance for a sub-region to be considered salient
        regions : the coordinates associated with each quadrant
        Returns
        --------
        srs : a list of the coordinates of the quadrants whose Shapley values were large enough
        """
        srs = []
        for i in range(len(shapley_values)):
            for j in range(len(shapley_values[0])):
                if shapley_values[i, j] > tol:
                    srs.append(regions[i, j])
        return srs

    def display_salient(self, im, srs_coll, count, filename ):
        """
        Determine which of the 4 quadrants are salient, i.e. have Shapley value larger than tol
        Parameters
        ----------
        im : the original image, in input format
        srs_coll : a collection of all regions deemed salient
        count : a normalizing mask which determines how many time each pixel was given a chance to be salient
        filename : name of the file to save the figure to
        """
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(45, 30))
        sample_image = im.numpy().transpose(1, 2, 0)
        count = count.transpose(1, 2, 0)
        image = sample_image * self.sd + self.mean
        ax1.imshow(image)
        ax2.imshow(image)
        mask = np.zeros(image.shape)

        # Count how many time each pixel was found to be in a salient region
        for srs in srs_coll:
            for sr in srs:
                start = sr[0]
                q_size = sr[1]

                xs = [start[1], start[1] + q_size[1], start[1] + q_size[1], start[1]]
                ys = [start[0], start[0], start[0] + q_size[0], start[0] + q_size[0]]
                ax2.fill(xs, ys, 'r', alpha=1 / len(srs_coll))
                mask[start[0]:start[0] + q_size[0], start[1]:start[1] + q_size[1], :] += np.ones(
                    (q_size[0], q_size[1], 3))

        # Normalize the mask by the number of tries in each region
        mask /= count
        # Normalize the mask to the range (0,1)
        mask /= np.max(mask)
        # Set to 0 elements smaller than 1/3
        negligible = (mask < 1/3)
        mask[negligible] = 0

        ax1.set_xlim([0, im.shape[2]])
        ax1.set_ylim([im.shape[1], 0])
        ax2.set_xlim([0, im.shape[2]])
        ax2.set_ylim([im.shape[1], 0])
        ax3.imshow(image * mask)
        if filename != None:
            plt.savefig(filename, dpi=300)
        return mask

    def do_all(self, im, label, start, region_size, tol, debug=False):
        """
        Secondary main loop: do everything for one region of the image.
        ----------
        im : the input image
        start : the starting coordinates of the region
        region_size : self-explanatory
        tol : the specified tolerance for a sub-region to be considered salient
        debug : if True, all subsets, there associated scores and the Shapley values will be displayed
        Returns
        --------
        srs : a list of the coordinates of the quadrants whose Shapley values were large enough
        """
        images_final, regions = self.construct_subsets(im, start, region_size)
        score = self.subset_scores(images_final, label)
        sm = self.shapley_of_quadrants(score)
        if debug:
            self.display_cropped_images(images_final, score)
            f = plt.figure()
            sns.heatmap(sm)
            f.suptitle("Shap values of each quadrant")

        srs = self.get_salient_regions(sm, tol, regions)

        return srs

    def saliency_map(self, image, label, tolerance, only_one_run=False, debug=False, max_depth=30, filename=None):
        """
        Create and then show a saliency map built with the Hierarchical Shapley method.
        ----------
        im : the input image
        label : the label with respect to which we want to analyze - typically 1
        tolerance : the specified tolerance for a sub-region to be considered salient. A list is expected.
        only_one_run : when False, several runs are done by also considering 16 cropped versions of the input
        debug : if True, all subsets, there associated scores and the Shapley values will be displayed
        max_depth : the maximum number of divisions you want to allow before deciding the tolerance is too low.
        filename : name of the file to save the figure to
        """
        ls = []
        count = np.ones(image.shape)
        xf = [image.shape[1], image.shape[2]]


        start = (0,0)
        end = (xf[0], xf[1])
        size = (end[0] - start[0], end[1] - start[1])
        lx, ly = image.shape[1], image.shape[2]
        dx, dy = image.shape[1]//4, image.shape[2]//4
       
        def salient_regions(I, sx, sy): 
          srs = [(start, size)]
          finished = []
          k = 0
          while len(srs) > 0:
            all_ = []
            for sr in srs:
                s = self.do_all(I, label, sr[0], sr[1], tolerance, debug)
                if s == []:

                    coords = [sr[0][0] + sx, sr[0][1] + sy]
                    finished.append(((coords), (sr[1])))
                else:
                    all_ += s
            srs = all_
            k += 1
          return finished

        


        # normal
        ls.append(salient_regions(image, 0, 0))
        
        # shifted to bottom right 
        image_br = self.background.clone()
        image_br[:, :lx-dx, :ly-dy] = image[:, dx:, dy:]
        ls.append(salient_regions(image_br, dx, dy))
          
         # shifted to bottom left 
        image_bl = self.background.clone()
        image_bl[:, :lx-dx, dy:] = image[:, dx:, :ly-dy]
        ls.append(salient_regions(image_bl, dx, -dy))

        # shifted to top left 
        image_tl = self.background.clone()
        image_tl[:, dx:, dy:] = image[:, :lx-dx, :ly-dy]
        ls.append(salient_regions(image_tl, -dx, -dy))

        # shifted to top right 
        image_tr = self.background.clone()
        image_tr[:, dx:, :ly-dy] = image[:, :lx-dx, dy:]
        ls.append(salient_regions(image_tr, -dx, dy))


        return self.display_salient(image, ls, count, filename)


average = torch.mean(X, dim = 0)
h2 = HierarchicalShap2(net, background = average)

ModuleNotFoundError: No module named 'seaborn'

In [19]:

h2.saliency_map(image1, label1, tolerance = [6], only_one_run = False);


NameError: name 'h2' is not defined

In [20]:
h.saliency_map(image1, label1, tolerance = [6], only_one_run = False);

NameError: name 'h' is not defined

In [21]:
def saliency_map(self, image, label, tolerance, only_one_run=False, debug=False, max_depth=30, filename=None):
        """
        Create and then show a saliency map built with the Hierarchical Shapley method.
        ----------
        im : the input image
        label : the label with respect to which we want to analyze - typically 1
        tolerance : the specified tolerance for a sub-region to be considered salient. A list is expected.
        only_one_run : when False, several runs are done by also considering 16 cropped versions of the input
        debug : if True, all subsets, there associated scores and the Shapley values will be displayed
        max_depth : the maximum number of divisions you want to allow before deciding the tolerance is too low.
        filename : name of the file to save the figure to
        """
        ls = []
        count = np.ones(image.shape)
        xf = [image.shape[1], image.shape[2]]


        start = (0,0)
        end = (xf[0], xf[1])
        size = (end[0] - start[0], end[1] - start[1])
        lx, ly = image.shape[1], image.shape[2]
        dx, dy = image.shape[1]//4, image.shape[2]//4
       
        def salient_regions(I, sx, sy): 
          srs = [(start, size)]
          finished = []
          k = 0
          while len(srs) > 0:
            all_ = []
            for sr in srs:
                s = self.do_all(I, label, sr[0], sr[1], tolerance, debug)
                if s == []:
                    coords = [sr[0][0] + sx, sr[0][1] + sy]
                    finished.append(((coords), (sr[1])))
                else:
                    all_ += s
                srs = all_
                k += 1
            return finished

        


        # normal
        ls.append(salient_regions(image, 0, 0))
        
        # shifted to bottom right 
        image_br = self.background.clone()
        image_br[:, :lx-dx, :ly-dy] = image[:, dx:, dy:]
        ls.append(salient_regions(image_br, dx, dy))
          
         # shifted to bottom left 
        image_bl = self.background.clone()
        image_bl[:, :lx-dx, dy:] = image[:, dx:, :ly-dy]
        ls.append(salient_regions(image_bl, dx, -dy))

        # shifted to top left 
        image_tl = self.background.clone()
        image_tl[:, dx:, dy:] = image[:, :lx-dx, :ly-dy]
        ls.append(salient_regions(image_tl, -dx, -dy))

        # shifted to top right 
        image_tr = self.background.clone()
        image_tr[:, dx:, :ly-dy] = image[:, :lx-dx, dy:]
        ls.append(salient_regions(image_tr, -dx, dy))


        return self.display_salient(image, ls, count, filename)

In [22]:
train_data = ImageFolder(root = os.path.join(root_dir, 'train'), transform = transf)
dataloader = DataLoader(train_data, batch_size = Batch_Size, shuffle = True, num_workers = 0)
train_loader = iter(dataloader)

images, labels = next(train_loader)
i = 0

for i in range(20):
  image = images[i]
  label = labels[i]

  input_ = image.view(-1, 3, 100, 120) # This shape is necessary for the network 
  output = net(input_)
  _, predicted = torch.max(output.data, 1)
  a = output.data[0][1].detach().numpy()
  img = utils.input2image(image, MEAN, STD)
  utils.display_image(img, a, int(predicted))
  

  if (a > 0):
    h.saliency_map(image, label, tolerance = l, only_one_run = False)
  i += 1

NameError: name 'os' is not defined